In [6]:
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt


In [1]:




def makewieghts(params):
    l = params["l"]
    coef = params["coef"]

    num = 1/2 - np.sqrt(5) / 2
    k = lambda x: 1/(x - num) + num


    def f(x):
        if k(x) > l:
            return l
        else:
            return k(x)
        
    wieghts = [0,0,0]

    wieghts[0] =f(coef)
    wieghts[1] = (f(coef) + f(1-coef)) / 2
    wieghts[2] = f(1 - coef)
    
    return wieghts


In [2]:
def readUserData(idUser):
    with open("data/" + str(idUser) + ".txt") as f:
        temp = "".join(f.readlines())
    return temp


def getUserCoef(idUser,params):


    try:
        with open("usersSettings/" + str(idUser) + ".txt", "r") as f:
            params["coef"] = float(f.readlines()[0])

    except:

        with open("usersSettings/" + str(idUser) + ".txt", "w") as f:
            f.writelines(str(params["coef"]))
            
    return params




In [3]:
def readUserData(temp):

    try:
        keys = temp[1:-1].split(" ")[::2]
        values = temp[1:-1].split(" ")[1::2]
    except:
        
        keys =  temp.keys()
        values = temp.values()

    keys = list(map(lambda x:x, keys))



    f = lambda x: list(map(int,list(filter(lambda x:x != "",x.split(",")))))

    def convertList(x):
        if len(x) > 1:
            return x
        else:
            return x[0]

    values = list(map(lambda x: f(x), values))
    userdata = dict(zip(keys,values))
    
    return userdata




In [26]:
def writeProcessDataUser(userdata,params, idUser):


    tempuserdata = dict(userdata)

    try:
        del tempuserdata['posts-read']
    except:
        pass
    
    try:
        del tempuserdata['posts-visited']
    except:
        pass

    #try:
    tagsuser = pd.DataFrame(tempuserdata).T
    #except:
     #   return tempuserdata
    
    tagsuser.columns = ["watch"]

    tagsuser["tags"] = tagsuser.index
    tagsuser["categ"] = tagsuser["tags"].map(lambda x: x.split("-")[0])
    tagsuser["tags"] = tagsuser["tags"].map(lambda x: x.split("-")[2])
    tagsuser.index = range(len(tagsuser))

    tagsuser.groupby(["tags"],as_index=False).mean()



    tagsuser["categ"] = tagsuser["categ"].map(dict(zip(["read","visit","view"],
                                   params["wieghts"])))

    tagsuser["temp"] = tagsuser["watch"] * tagsuser["categ"]
    r = tagsuser.groupby(["tags"])["temp"].sum()
    

    finalUserTags = r / tagsuser.groupby(["tags"])["watch"].sum()

    finalUserTags.sort_values(ascending=False)
    


    try:
        t = pd.DataFrame(finalUserTags)
    except:

        return t
    
    t.columns = ["score"]

    
    return t
    



In [ ]:
    
    
#d = pd.read_csv("data/users_export_onesignal_users_2019-05-11T16_43_12+00_00.csv")
  

In [95]:
#readUserData(d["tags"][1])

def makeData(idUser, d, coef):
    params = {}

    params["l"] = 0.5
    params["coef"] = coef


    params["wieghts"] = makewieghts(params)
    

    

    temp = json.loads(d["tags"][idUser])
    #return temp
    
    
    if(temp == {}):
        raise ValueError


    userdata = readUserData(temp)
    #return userdata


    t = writeProcessDataUser(userdata,params, str(idUser))

    #t.to_csv("clientTags/"+str(idUser)+".csv",index=True)
    return t
  
t = makeData(300, d, 0.3)
t

,score
tags,
blockchain,0.305961
courses,0.140672
google,0.239845
guest,0.305961
learn,0.239845
machine,0.140672
microsoft,0.140672
open,0.239845


,score
tags,
blockchain,0.270356
courses,0.040713
google,0.178499
guest,0.270356
learn,0.178499
machine,0.040713
microsoft,0.040713
open,0.178499


In [92]:
def makeLink(t, r):
    col = list(set(r.columns).intersection(set(t.index)))
    table = r[col].applymap(int)



    tableCoef = pd.DataFrame((np.array(table) * np.array(t.T[col])).T).T
    tableCoef.columns = col

    tableCoef["score"] = tableCoef.sum(axis=1)
    tableCoef["link"] = r["ids"]


    tableCoef = tableCoef.sort_values("score", ascending=False)
    return tableCoef[tableCoef["score"] > 0]


r = pd.read_csv("data/tagsConv.csv")
makeLink(t, r)

,google,courses,guest,blockchain,score,link
145,0.000000,0.000000,0.000000,0.270356,0.270356,https://tproger.ru/?p=31444
125,0.000000,0.000000,0.270356,0.000000,0.270356,https://tproger.ru/?p=31842
52,0.000000,0.000000,0.270356,0.000000,0.270356,https://tproger.ru/?p=34989
44,0.000000,0.000000,0.270356,0.000000,0.270356,https://tproger.ru/?p=32642
157,0.178499,0.040713,0.000000,0.000000,0.219212,https://tproger.ru/?p=65227
107,0.178499,0.000000,0.000000,0.000000,0.178499,https://tproger.ru/?p=70752
111,0.178499,0.000000,0.000000,0.000000,0.178499,https://tproger.ru/?p=35085
158,0.178499,0.000000,0.000000,0.000000,0.178499,https://tproger.ru/?p=33892
73,0.178499,0.000000,0.000000,0.000000,0.178499,https://tproger.ru/?p=24855
173,0.178499,0.000000,0.000000,0.000000,0.178499,https://tproger.ru/?p=81187


tags,google,courses,guest,blockchain
score,0.178499,0.040713,0.270356,0.270356


,google,courses,guest,blockchain,score,link
145,0.000000,0.000000,0.000000,0.270356,0.270356,https://tproger.ru/?p=31444
125,0.000000,0.000000,0.270356,0.000000,0.270356,https://tproger.ru/?p=31842
52,0.000000,0.000000,0.270356,0.000000,0.270356,https://tproger.ru/?p=34989
44,0.000000,0.000000,0.270356,0.000000,0.270356,https://tproger.ru/?p=32642
157,0.178499,0.040713,0.000000,0.000000,0.219212,https://tproger.ru/?p=65227
107,0.178499,0.000000,0.000000,0.000000,0.178499,https://tproger.ru/?p=70752
111,0.178499,0.000000,0.000000,0.000000,0.178499,https://tproger.ru/?p=35085
158,0.178499,0.000000,0.000000,0.000000,0.178499,https://tproger.ru/?p=33892
73,0.178499,0.000000,0.000000,0.000000,0.178499,https://tproger.ru/?p=24855
173,0.178499,0.000000,0.000000,0.000000,0.178499,https://tproger.ru/?p=81187
